In [1]:
application "tropical";

In [2]:
sub harnack_curve_tmp {
  my $n = shift;
  my @monoms;
  my @weights;
  my @signs;
  foreach my $i (0 .. $n) {
    foreach my $j (0 .. $n-$i) {
      push @weights, ($i*$j+$i*$i+$j*$j);
      push @monoms, [$n-$i-$j, $i, $j];
      push @signs, ($i*$j+$i+$j)%2;
    }
  }

  my $m = new Matrix<Int>(\@monoms);
  my $c = new Vector<TropicalNumber<Min>>(\@weights);
  my $s = new Array<Bool>(\@signs);
  my $h = new Hypersurface<Min>(MONOMIALS=>$m, COEFFICIENTS=>$c);
  $h->PATCHWORK(SIGNS=>$s);
  return $h;
}

In [3]:
$h = harnack_curve_tmp(5);

In [4]:
$h->properties;

type: Hypersurface<Min>

COEFFICIENTS
0 1 4 9 16 25 1 3 7 13 21 4 7 12 19 9 13 19 16 21 25

MONOMIALS
5 0 0
4 0 1
3 0 2
2 0 3
1 0 4
0 0 5
4 1 0
3 1 1
2 1 2
1 1 3
0 1 4
3 2 0
2 2 1
1 2 2
0 2 3
2 3 0
1 3 1
0 3 2
1 4 0
0 4 1
0 5 0


PATCHWORK
type: Patchwork

PURE
true


In [5]:
$comp = $h->COMPACTIFICATION;

Click here for additional output 
 
polymake: used package cdd
 cddlib
 Implementation of the double description method of Motzkin et al.
 Copyright by Komei Fukuda.
 http://www-oldurls.inf.ethz.ch/personal/fukudak/cdd_home/

In [6]:
$rf = $h->PATCHWORK->REAL_FACETS;

In [7]:
print $rf;

{0 1 2 3 4 5 8 10 14 16 19 20 23 24 25 26 29 30 33 35 38 39 42 43}
{0 1 2 3 4 6 7 11 13 17 21 22 23 26 27 28 29 31 32 36 40 41 42 44}
{5 6 9 10 11 12 15 16 17 18 19 22 24 28 30 31 34 35 36 37 38 41 43 44}
{7 8 9 12 13 14 15 18 20 21 25 27 32 33 34 37 39 40}


In [8]:
print rows_labeled($h->MAXIMAL_POLYTOPES);

0:0 13
1:0 14
2:0 16
3:0 5
4:0 1
5:13 26
6:12 13
7:12 14
8:14 15
9:11 12
10:15 16
11:7 16
12:9 15
13:5 7
14:3 5
15:6 7
16:1 3
17:1 27
18:2 3
19:11 26
20:10 11
21:9 10
22:9 17
23:8 10
24:6 17
25:4 6
26:17 19
27:2 4
28:2 27
29:4 22
30:8 26
31:8 18
32:18 19
33:19 21
34:18 20
35:21 22
36:22 27
37:21 24
38:20 26
39:20 23
40:23 24
41:24 27
42:23 25
43:25 26
44:25 27


In [11]:
print rows_labeled($h->COMPACTIFICATION->DECORATION);

0:{} 0 {} {}
1:{0} 1 {0 13} {0}
2:{1} 1 {0 14} {0}
3:{2} 1 {0 16} {0}
4:{3} 1 {0 5} {0}
5:{4} 1 {0 1} {0}
6:{5} 1 {13 26} {26}
7:{6} 1 {1 27} {27}
8:{7} 1 {11 26} {26}
9:{8} 1 {2 27} {27}
10:{9} 1 {8 26} {26}
11:{10} 1 {22 27} {27}
12:{11} 1 {20 26} {26}
13:{12} 1 {24 27} {27}
14:{13} 1 {25 26} {26}
15:{14} 1 {25 27} {27}
16:{15} 1 {13} {}
17:{16} 1 {14} {}
18:{17} 1 {16} {}
19:{18} 1 {5} {}
20:{19} 1 {1} {}
21:{20} 1 {12} {}
22:{21} 1 {15} {}
23:{22} 1 {11} {}
24:{23} 1 {7} {}
25:{24} 1 {9} {}
26:{25} 1 {3} {}
27:{26} 1 {6} {}
28:{27} 1 {2} {}
29:{28} 1 {10} {}
30:{29} 1 {17} {}
31:{30} 1 {8} {}
32:{31} 1 {4} {}
33:{32} 1 {19} {}
34:{33} 1 {22} {}
35:{34} 1 {18} {}
36:{35} 1 {21} {}
37:{36} 1 {20} {}
38:{37} 1 {24} {}
39:{38} 1 {23} {}
40:{39} 1 {25} {}
41:{0 15} 2 {0 13} {}
42:{1 16} 2 {0 14} {}
43:{2 17} 2 {0 16} {}
44:{3 18} 2 {0 5} {}
45:{4 19} 2 {0 1} {}
46:{5 15} 2 {13 26} {}
47:{6 19} 2 {1 27} {}
48:{7 22} 2 {11 26} {}
49:{8 27} 2 {2 27} {}
50:{9 30} 2 {8 26} {}
51:{10 33} 2 {2

In [12]:
print $comp->nodes_of_rank(2)->size();

45

In [13]:
$decor = $comp->DECORATION;
$facets = $h->MAXIMAL_POLYTOPES;
$real_facets = $h->PATCHWORK->REAL_FACETS;
$nm = new NodeMap<Directed, Set<Int>>($comp->ADJACENCY);
foreach my $node (@{$comp->nodes_of_rank(2)}){
    my $real = $decor->[$node]->realisation;
    # Only works since the thing is pure
    my $facetno = -1;
    for(my $i=0; $i<$facets->rows; $i++){
        if($facets->[$i] == $real){
            $facetno = $i;
            last;
        }
    }
    print $node," ",$real," ",$facetno,": ";

    for(my $i = 0; $i<$real_facets->rows; $i++){
        if($real_facets->[$i]->contains($facetno)){
            print $i,",";
            $nm->[$node] += $i;
        }
    }
    print "\n";
    
}

41 {0 13} 0: 0,1,
42 {0 14} 1: 0,1,
43 {0 16} 2: 0,1,
44 {0 5} 3: 0,1,
45 {0 1} 4: 0,1,
46 {13 26} 5: 0,2,
47 {1 27} 17: 1,2,
48 {11 26} 19: 0,2,
49 {2 27} 28: 1,2,
50 {8 26} 30: 0,2,
51 {22 27} 36: 1,2,
52 {20 26} 38: 0,2,
53 {24 27} 41: 1,2,
54 {25 26} 43: 0,2,
55 {25 27} 44: 1,2,
56 {12 13} 6: 1,2,
57 {12 14} 7: 1,3,
58 {14 15} 8: 0,3,
59 {15 16} 10: 0,2,
60 {7 16} 11: 1,2,
61 {5 7} 13: 1,3,
62 {3 5} 14: 0,3,
63 {1 3} 16: 0,2,
64 {11 12} 9: 2,3,
65 {9 15} 12: 2,3,
66 {10 11} 20: 0,3,
67 {6 7} 15: 2,3,
68 {9 10} 21: 1,3,
69 {9 17} 22: 1,2,
70 {2 3} 18: 2,3,
71 {6 17} 24: 0,2,
72 {4 6} 25: 0,3,
73 {2 4} 27: 1,3,
74 {8 10} 23: 0,1,
75 {17 19} 26: 0,1,
76 {8 18} 31: 1,2,
77 {4 22} 29: 0,1,
78 {18 19} 32: 1,3,
79 {19 21} 33: 0,3,
80 {21 22} 35: 0,2,
81 {18 20} 34: 2,3,
82 {21 24} 37: 2,3,
83 {20 23} 39: 0,3,
84 {23 24} 40: 1,3,
85 {23 25} 42: 0,1,


In [14]:
foreach my $node (@{$comp->nodes_of_rank(1)}){
    # print $comp->ADJACENCY->out_adjacent_nodes($node),"\n";
    foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
        $nm->[$node] += $nm->[$parent];
    }
}
foreach my $node (@{$comp->nodes_of_rank(0)}){
    # print $comp->ADJACENCY->out_adjacent_nodes($node),"\n";
    foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
        $nm->[$node] += $nm->[$parent];
    }
}

In [15]:
$em = new EdgeMap<Directed, Matrix<Integer>>($comp->ADJACENCY);

In [16]:
sub make_matrix {
    my($source, $target) = @_;
    # print "S: ",$source,"T: ",$target,"\n";
    my $result = unit_matrix<Integer>($source->size());
    my $remaps = new Set<Int>();
    my $remapt = new Set<Int>();
    my $i = 0;
    foreach my $s (@$source){
        $remaps += $i;
        if($target->contains($s)){
            $remapt += $i;
        }
        $i++;
    }
    return $result->minor($remapt, $remaps);
}

In [17]:
for (my $e=entire(edges($comp->ADJACENCY)); $e; ++$e) {
    $em->[$$e] = make_matrix($nm->[$e->from_node], $nm->[$e->to_node]);
}

In [18]:
print $comp->TOP_NODE;

86

In [19]:
$top = $comp->TOP_NODE;
foreach my $node (@{$comp->ADJACENCY->in_adjacent_nodes($top)}){
    $em->edge($node, $top) = zero_matrix<Integer>($nm->[$node]->size(),0);
}

In [21]:
$chain = fan::build_full_chain($comp, $comp->ORIENTATIONS, $em, true);

Click here for additional output 
 
polymake: WARNING: Compiling temporary shared module, please be patient...

In [ ]:
print $comp->ORIENTATIONS;

In [22]:
print $chain;

<1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

In [24]:
print $chain->type->full_name;

ChainComplex<Matrix<Integer, NonSymmetric>>

In [23]:
print topaz::betti_numbers($chain);

14 24

Click here for additional output 
 
polymake: WARNING: Compiling temporary shared module, please be patient...

In [ ]:
$hr = $h->PATCHWORK->realize("uniform");

In [ ]:
print $hr->HOMOLOGY;

In [ ]:
$chainI = new topaz::ChainComplex<Matrix<Integer>>($chain);